In [1]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 1

In [2]:
from cast.models import (
    Blog as CastBlog,
    Image,
    Video,
    File,
    Post,
    Gallery,
)

In [3]:
from homepage.blogs.models import (
    Blog,
    BlogImage,
    BlogFile,
    BlogGallery,
    BlogPost,
    BlogVideo,
)

# Migrate Blogs

In [4]:
CastBlog.objects.all().delete()

(0, {})

In [96]:
%%time
blog_attrs = ("pk", "user", "title", "description", "slug", "created", "modified")
cast_blogs = []
for bl in Blog.objects.all():
    params = {attr: getattr(bl, attr) for attr in blog_attrs}
    cast_blogs.append(CastBlog(**params))

CPU times: user 3.35 ms, sys: 1.42 ms, total: 4.77 ms
Wall time: 3.58 ms


In [97]:
%%time
created = CastBlog.objects.bulk_create(cast_blogs)

CPU times: user 1.48 ms, sys: 1.45 ms, total: 2.93 ms
Wall time: 40.9 ms


In [98]:
CastBlog.objects.count()

2

# Migrate Images

In [99]:
Image.objects.all().delete()

(1077, {'cast.Gallery_images': 507, 'cast.Post_images': 0, 'cast.Image': 570})

In [100]:
%%time
image_attrs = ("pk", "original", "original_height", "original_width", "user", "created", "modified")
cast_images = []
for bi in BlogImage.objects.all():
    params = {attr: getattr(bi, attr) for attr in image_attrs}
    cast_images.append(Image(**params))

CPU times: user 406 ms, sys: 11.8 ms, total: 418 ms
Wall time: 495 ms


In [101]:
%%time
created = Image.objects.bulk_create(cast_images)

CPU times: user 76.6 ms, sys: 2.93 ms, total: 79.6 ms
Wall time: 102 ms


In [102]:
Image.objects.count()

570

# Migrate Videos

In [103]:
Video.objects.all().delete()

(49, {'cast.Post_videos': 0, 'cast.Video': 49})

In [104]:
%%time
video_attrs = ("pk", "original", "poster", "poster_seconds", "user", "created", "modified")
cast_videos = []
for bv in BlogVideo.objects.all():
    params = {attr: getattr(bv, attr) for attr in video_attrs}
    cast_videos.append(Video(**params))

CPU times: user 41.4 ms, sys: 2.67 ms, total: 44.1 ms
Wall time: 50.1 ms


In [105]:
%%time
created = Video.objects.bulk_create(cast_videos)

CPU times: user 4.93 ms, sys: 1.15 ms, total: 6.09 ms
Wall time: 86.6 ms


In [106]:
Video.objects.count()

49

# Migrate Files

In [107]:
File.objects.all().delete()

(1, {'cast.File': 1})

In [108]:
%%time
file_attrs = ("pk", "original", "user", "created", "modified")
cast_files = []
for bf in BlogFile.objects.all():
    params = {attr: getattr(bf, attr) for attr in file_attrs}
    cast_files.append(File(**params))

CPU times: user 2.29 ms, sys: 822 µs, total: 3.11 ms
Wall time: 2.37 ms


In [109]:
%%time
created = File.objects.bulk_create(cast_files)

CPU times: user 1.17 ms, sys: 1.34 ms, total: 2.51 ms
Wall time: 41.3 ms


In [110]:
File.objects.count()

1

# Migrate Galleries

In [111]:
Gallery.objects.all().delete()

(66, {'cast.Gallery_images': 0, 'cast.Post_galleries': 0, 'cast.Gallery': 66})

In [112]:
%%time
gallery_attrs = ("pk", "user", "created", "modified")
for bg in BlogGallery.objects.all():
    params = {attr: getattr(bg, attr) for attr in gallery_attrs}
    gallery = Gallery.objects.create(**params)
    img_pks = [pk for pk, in bg.images.values_list("pk")]
    images = Image.objects.filter(pk__in=img_pks)
    for image in images:
        gallery.images.add(image)
    gallery.save()

CPU times: user 788 ms, sys: 45 ms, total: 833 ms
Wall time: 2.65 s


In [113]:
Gallery.objects.count()

66

# Migrate Posts

In [203]:
Post.objects.all().delete()

(7,
 {'cast.Post_images': 4,
  'cast.Post_videos': 0,
  'cast.Post_galleries': 2,
  'cast.Post': 1})

In [204]:
%%time
post_attrs = ("pk", "author", "title", "pub_date", "visible_date",
              "content", "slug", "created", "modified")
for bp in BlogPost.objects.all():
    params = {attr: getattr(bp, attr) for attr in post_attrs}
    params["content"] = (params["content"].replace("blog_image", "image")
                                          .replace("blog_gallery", "gallery")
                                          .replace("blog_video", "video"))
    params["blog"] = CastBlog.objects.get(pk=bp.blog.pk)
    post = Post.objects.create(**params)
    
    img_pks = [pk for pk, in bp.images.values_list("pk")]
    images = Image.objects.filter(pk__in=img_pks)
    print(images)
    for image in images:
        post.images.add(image)
    print(post.images.all())
    print(post)
    #post.save()
    #print(post.images.all())
    video_pks = [pk for pk, in bp.videos.values_list("pk")]
    videos = Video.objects.filter(pk__in=video_pks)
    for video in videos:
        post.videos.add(video)

    gallery_pks = [pk for pk, in bp.galleries.values_list("pk")]
    galleries = Gallery.objects.filter(pk__in=gallery_pks)
    for gallery in galleries:
        post.galleries.add(gallery)
    
    #post.save()

<QuerySet [<Image: blogs_images/originals/fullsizeoutput_3164.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_3164.jpeg>]>
Zu ersten Mal etwas Möhrenbrei
<QuerySet []>
<QuerySet []>
Zu Besuch bei den Grosseltern
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_31b9.jpeg>, <Image: blogs_images/originals/fullsizeoutput_31ba.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_31b9.jpeg>, <Image: blogs_images/originals/fullsizeoutput_31ba.jpeg>]>
Tapas mit Heike und Chrissie
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_31bd.jpeg>, <Image: blogs_images/originals/fullsizeoutput_31bc.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_31bd.jpeg>, <Image: blogs_images/originals/fullsizeoutput_31bc.jpeg>]>
Martin und Marcus kommen zu Besuch
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_328c.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_328c.jpeg>]>
Kaffeetrinken in der Spoerl-Fabrik
<QuerySet [<Image: b

<QuerySet [<Image: blogs_images/originals/eeK81WRQaCrETf1Xz3Ug.jpg>, <Image: blogs_images/originals/NRgRHnUKShCleAHpFOnsg.jpg>]>
Spaziergang in Düsseldorf
<QuerySet []>
<QuerySet []>
Wintergarten wieder ohne Kissen
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_39f4.jpeg>, <Image: blogs_images/originals/fullsizeoutput_39f5.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_39f4.jpeg>, <Image: blogs_images/originals/fullsizeoutput_39f5.jpeg>]>
Karfreitag im Carrotcake
<QuerySet []>
<QuerySet []>
Zum ersten Mal beim Krabbeln gefilmt
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_3a38.jpeg>, <Image: blogs_images/originals/fullsizeoutput_3a01.jpeg>, <Image: blogs_images/originals/fullsizeoutput_3a2c.jpeg>, <Image: blogs_images/originals/fullsizeoutput_3a27.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_3a38.jpeg>, <Image: blogs_images/originals/fullsizeoutput_3a01.jpeg>, <Image: blogs_images/originals/fullsizeoutput_3a2c.jpeg>, <Image: bl

<QuerySet [<Image: blogs_images/originals/fullsizeoutput_3e1f.jpeg>]>
<QuerySet [<Image: blogs_images/originals/fullsizeoutput_3e1f.jpeg>]>
Claas erste Demo
<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>
<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>
Picknick vor Hochofen 5
CPU times: user 1.88 s, sys: 154 ms, total: 2.03 s
Wall time: 7.23 s


In [196]:
post.images.all()

<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>

In [146]:
Post.objects.count()

117

In [121]:
post = Post.objects.get(slug="pycon_2017")

In [140]:
foo = post.content

In [132]:
post.pk

27

In [131]:
BlogPost.objects.all()[1].description

AttributeError: 'BlogPost' object has no attribute 'description'

In [133]:
bp = BlogPost.objects.get(pk=27)

In [135]:
bp.content

'<a href="https://de.pycon.org/">PyCon 2017</a>&nbsp;is over now and I liked it a lot. It was a really well organized event and the talks were great. I already knew the <a href="http://zkm.de/en">conference location</a>&nbsp;because I lived only a few hundred meters away for a long time. So it didn&#39;t came as a surprise for me that it was awesome.<br />\r\n<br />\r\n{% blog_gallery 8 %}<br />\r\n<br />\r\nI don&#39;t attend conferences like this because of the talks, I have to say. But I always get a strange boost in motivation from events like this - I don&#39;t now why, but I suspect it has something to do with talking to a lot of likeminded people. Talking to my former collegues from&nbsp;<a href="https://www.billiger.de/">billiger.de</a>&nbsp;for example was a lot of fun. They did a lot of cool things since I moved on like having a new search service in pure python or migrating from relational databases to redis and now to elasticsearch for document storage.<br />\r\n<br />\r\n{

In [136]:
bp.description

AttributeError: 'BlogPost' object has no attribute 'description'

In [137]:
Post.objects.count()

0

In [143]:
foo.replace("blog_image", "image").replace("blog_gallery", "gallery").replace()

'<h3>Besuch im Landschaftspark Duisburg-Nord</h3>\r\n{% image 529 %}<br />\r\n{% image 537 %}<br />\r\n<br />\r\nAuf dem riesigen Gel&auml;nde des ehemaligen H&uuml;ttenwerks in Duisburg gibt es wirklich viel zu entdecken. Es wird sicher spannend in der Anlage herumklettern zu k&ouml;nnen, wenn Claas ein paar Jahre &auml;lter ist. Aber auch zu ebener Erde ist der Anblick der vor sich hinrostenden Hoch&ouml;fen und der Bunkerg&auml;rten einen Besuch wert. Die &quot;Alte Emscher&quot; flie&szlig;t durch das Gel&auml;nde und in den ehemaligen Absetzbecken wuchern die Seerosen. Kurz, dieser Park hat uns nicht zum letzten Mal gesehen.&nbsp;<br />\r\n{% gallery 76 %}<br />\r\nNach unserem Picknick vor Hochofen 5, haben wir mit Claas einige der oberen Steige zu Fu&szlig; erkundet. Daf&uuml;r, dass er noch nicht sehr lange l&auml;uft hat er die langen Wege wirklich super gemeistert. V&ouml;llig angstfrei &uuml;ber Gitterbr&uuml;cken und so begeistert von den Bodenwellen auf einem offenen Platz

In [147]:
Image.objects.get(pk=527)

<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-1.jpg>

In [202]:
Post.objects.get(pk=124).images.all()

<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>

In [197]:
bp = BlogPost.objects.get(pk=124)

In [153]:
img_pks = [pk for pk, in bp.images.values_list("pk")]

In [154]:
img_pks

[529, 533, 537, 566]

In [155]:
images = Image.objects.filter(pk__in=img_pks)

In [156]:
images

<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>

In [205]:
post = Post.objects.get(pk=124)

In [159]:
for image in images:
    post.images.add(image)

In [160]:
post.images.all()

<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>

In [206]:
post.images.all()

<QuerySet [<Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-3.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-7.jpg>, <Image: blogs_images/originals/Industriepark-Duisburg-Juli-2018-11.jpg>, <Image: blogs_images/originals/Strandbar_Duisburg.jpg>]>

In [207]:
post.created, post.modified

(datetime.datetime(2018, 8, 3, 17, 30, 54, 936869, tzinfo=<UTC>),
 datetime.datetime(2018, 11, 6, 15, 32, 5, 194251, tzinfo=<UTC>))

In [213]:
video = Video.objects.all().order_by("-created")[0]

In [214]:
video.created

datetime.datetime(2018, 8, 4, 8, 54, 29, 253890, tzinfo=<UTC>)

In [219]:
sorted([pk for pk, in Post.objects.values_list("pk")])

[1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 124,
 126]

In [224]:
Video.objects.get(pk=59)

DoesNotExist: Video matching query does not exist.

In [232]:
max([pk for pk, in Video.objects.values_list("pk")])

60

In [226]:
max([pk for pk, in BlogVideo.objects.values_list("pk")])

59

In [4]:
video = BlogVideo.objects.all().order_by("-created")[0]

In [5]:
video.poster

<ImageFieldFile: blogs_videos/poster/poster_s77g5s3o.jpg>

<ImageFieldFile: blogs_videos/poster/poster_s77g5s3o.jpg>

In [6]:
video.pk

59

59

In [7]:
video = Video.objects.all().order_by("-created")[0]

In [8]:
video.pk

60

60

In [9]:
video.poster

<ImageFieldFile: None>

<ImageFieldFile: None>

In [ ]:
video.create_poster()

/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_siswrc9x.jpg
> /Users/jochen/projects/django-cast/cast/models.py(158)_create_poster()
-> self.poster.save(
/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_siswrc9x.jpg
> /Users/jochen/projects/django-cast/cast/models.py(158)_create_poster()
-> self.poster.save(
(Pdb) import os
(Pdb) print(os.stat(tmp_path)
*** SyntaxError: unexpected EOF while parsing
*** SyntaxError: unexpected EOF while parsing
(Pdb) print(os.stat(tmp_path))
os.stat_result(st_mode=33152, st_ino=8627049245, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=285582, st_atime=1541579098, st_mtime=1541579098, st_ctime=1541579098)
os.stat_result(st_mode=33152, st_ino=8627049245, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=285582, st_atime=1541579098, st_mtime=1541579098, st_ctime=1541579098)
(Pdb) os.path.exists(tmp_path)
True
True
(Pdb) os.path.basename(tmp_path)
'poster_siswrc9x.jpg'
'poster_siswrc9x.jpg'


In [13]:
video.calc_poster

True

True

In [14]:
video._create_poster()

/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_oe5wtm5e.jpg
/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_oe5wtm5e.jpg


AttributeError: 'File' object has no attribute 'read'

AttributeError: 'File' object has no attribute 'read'

# move images

In [7]:
image = BlogImage.objects.all()[0]

In [8]:
image.original

<ImageFieldFile: blogs_images/originals/IMG_1773.JPG>

In [10]:
image = Image.objects.all()[0]

In [11]:
image.original

<ImageFieldFile: blogs_images/originals/IMG_1773.JPG>

In [13]:
image = Image.objects.get(pk=577)

In [14]:
image.original

<ImageFieldFile: cast_images/originals/iqHs41HGQsW2Ma1jN9iCag.jpg>

# video poster

In [4]:
import io
import os
import tempfile

from django.core.files import File
from subprocess import check_output

In [5]:
video = Video.objects.all().order_by("-created")[0]
print(video.pk)

60


In [6]:
video._create_poster()

In [24]:
content = video._get_content()

content:  File object (<_io.BufferedReader name='/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_fum4jsvc.jpg'>)


In [26]:
content.file

AttributeError: 'File' object has no attribute 'file'

In [28]:
content.size()

AttributeError: 'File' object has no attribute 'size'

In [9]:
content.file = open('/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_lj52okma.jpg', "rb")

In [11]:
name = "poster_lj52okma.jpg"

In [10]:
print(content)

File object (<_io.BufferedReader name='/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_lj52okma.jpg'>)


In [9]:
fp, tmp_path = tempfile.mkstemp(prefix='poster_', suffix='.jpg')

In [16]:
print(fp)

57


In [10]:
print(video.original.url)
print(video.original.path)
video_url = video.original.path

/media/blogs_videos/js7CDaxSx2sm0n3VanytQ_V4Sjx0r.MOV
/Users/jochen/projects/homepage/homepage/media/blogs_videos/js7CDaxSx2sm0n3VanytQ_V4Sjx0r.MOV


In [11]:
width, height = video._get_video_dimensions(video_url)

In [12]:
poster_cmd = (
    'ffmpeg -ss {seconds} -i "{video_path}" -vframes 1'
    ' -y -f image2 -s {width}x{height} {poster_path}'
).format(video_path=video_url, seconds=video.poster_seconds,
         poster_path=tmp_path, width=width, height=height)

In [13]:
poster_cmd

'ffmpeg -ss 1.0 -i "/Users/jochen/projects/homepage/homepage/media/blogs_videos/js7CDaxSx2sm0n3VanytQ_V4Sjx0r.MOV" -vframes 1 -y -f image2 -s 3840x2160 /var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg'

In [14]:
check_output(poster_cmd, shell=True)

b''

In [17]:
tmp_path

'/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg'

In [14]:
help(video.poster.save)

Help on method save in module django.db.models.fields.files:

save(name, content, save=True) method of django.db.models.fields.files.ImageFieldFile instance



In [18]:
name = os.path.basename(tmp_path)
print(name)

poster_67_qy0sc.jpg


In [19]:
content = File(open(tmp_path, 'rb'))
print("content: ", content)

content:  /var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg


In [20]:
fh = open(tmp_path, 'rb')
print(fh)

<_io.BufferedReader name='/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg'>


In [22]:
print(content)

/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg


In [23]:
content = File(fh)

In [25]:
print(repr(content))

<File: /var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg>


In [26]:
print(content.file)

<_io.BufferedReader name='/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_67_qy0sc.jpg'>


In [19]:
help(File)

Help on class File in module django.core.files.base:

class File(django.core.files.utils.FileProxyMixin)
 |  File(file, name=None)
 |  
 |  A mixin class used to forward file methods to an underlaying file
 |  object.  The internal file object has to be called "file"::
 |  
 |      class FileProxy(FileProxyMixin):
 |          def __init__(self, file):
 |              self.file = file
 |  
 |  Method resolution order:
 |      File
 |      django.core.files.utils.FileProxyMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exc_type, exc_value, tb)
 |  
 |  __init__(self, file, name=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  chunks(self, chunk_size=None)
 |      Read the file and yield chunks of ``chunk_size`` bytes (defa

In [7]:
help(open)

Help on built-in function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise OSError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position

In [12]:
video.poster.save(name, content, save=False)

AttributeError: 'File' object has no attribute 'read'

In [19]:
video.poster.field.max_length

100

In [20]:
video.poster.storage.save(name, content, max_length=video.poster.field.max_length)

AttributeError: 'File' object has no attribute 'read'

In [21]:
video.poster.storage._save(name, content)

AttributeError: 'File' object has no attribute 'chunks'

In [23]:
content.chunks

AttributeError: 'File' object has no attribute 'chunks'

In [15]:
content.file.read

<function BufferedReader.read(size=-1, /)>

In [22]:
video.poster

<ImageFieldFile: blogs_videos/poster/poster_j3ddhwqm.jpg>

In [21]:
video.save()

In [6]:
video._create_poster()

fh:  <_io.FileIO name='/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_fyr6w6c_.jpg' mode='rb' closefd=True>
content:  File object (<_io.FileIO name='/var/folders/2l/7zccp5bs1234yz4ns62s796m0000gn/T/poster_fyr6w6c_.jpg' mode='rb' closefd=True>)


AttributeError: 'File' object has no attribute 'read'

In [11]:
video.poster

<ImageFieldFile: blogs_videos/poster/poster_tq1kzqfv.jpg>

# resetting autoincrement

In [215]:
from django.db import connection

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("select max(id) from 